<a href="https://colab.research.google.com/github/aminfadaei116/Deep-Learning-Course/blob/master/Final%20Project/DL_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing the Required Pakages and Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/mcordts/cityscapesScripts.git
!pip install GPUtil
!pip install -q tf-nightly-2.0-preview

import pickle
import time
from cityscapesScripts.cityscapesscripts.helpers.labels import labels, name2label, id2label, trainId2label, category2labels
from PIL import Image
import numpy as np
from scipy.io import loadmat
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
from torchvision import transforms, datasets, models
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables import Keypoint, KeypointsOnImage
import csv
from torchvision.models.utils import load_state_dict_from_url
from collections import OrderedDict
import GPUtil as GPU
from collections import namedtuple
from torch.utils.tensorboard import SummaryWriter

import warnings
warnings.filterwarnings("ignore")

fatal: destination path 'cityscapesScripts' already exists and is not an empty directory.
ERROR: Could not find a version that satisfies the requirement tf-nightly-2.0-preview (from versions: none)
ERROR: No matching distribution found for tf-nightly-2.0-preview


# Start Tensorboard

In [ ]:
%load_ext tensorboard

logs_base_dir = "runs"
os.makedirs(logs_base_dir, exist_ok=True)

# Downloading the Cityscape Dataset

In [ ]:
%%bash

PACKAGE_IDS=(1, 3)

mkdir dataset

wget --keep-session-cookies --save-cookies=cookies.txt --post-data "username=arashito&password=arashito&submit=Login" https://www.cityscapes-dataset.com/login/

for ID in ${PACKAGE_IDS[@]}; do
  wget -O temp.zip --load-cookies cookies.txt --content-disposition "https://www.cityscapes-dataset.com/file-handling/?packageID=$ID"
  unzip temp.zip -d dataset
  rm temp.zip
done

rm index.html
rm cookies.txt

# Label Info

In [ ]:
desired_labels = ['road',
                  'sidewalk',
                  'parking',
                  'building',
                  'fence',
                  'wall',
                  'vegetation',
                  'terrain',
                  'car',
                  'bicycle',
                  'bus',
                  'truck',
                  'sky',
                  'person',
                  'pole',
                  'traffic sign',
                  'static',
                  'ground',
                  'dynamic']


Label = namedtuple('Label_info', ['name', 'original_id', 'id', 'color'])

label2info = {}

for id, label in enumerate(desired_labels):
    label2info[label] = Label(name=name2label[label].name, original_id=name2label[label].id, id=id+1, color=name2label[label].color)
label2info[None] = Label(name=name2label['unlabeled'].name, original_id=name2label['unlabeled'].id, id=0, color=name2label['unlabeled'].color)

# Processing Labels

In [ ]:
%%bash

cd /content/dataset
find . -name '*.json' -type f -delete
find . -name '*_color.png' -type f -delete
find . -name '*_instanceIds.png' -type f -delete

In [ ]:
ORIGINAL_LABELS = range(36)

label_map_dict = {label_info.original_id:label_info.id for label_info in label2info.values()}
unknown_label = label2info[None].id

for original_label in ORIGINAL_LABELS:
    if original_label not in label_map_dict:
        label_map_dict[original_label]=unknown_label


gt_path = '/content/dataset/gtFine'

counter=1
for root, dirs, files in os.walk(gt_path):
  for file in files:
      if file.endswith('labelIds.png'):
          print('\r', counter, end='')
          image_arr = np.array(Image.open(root+'/'+file))
          modified_img = np.vectorize(label_map_dict.get)(image_arr).astype(np.uint8)
          glob = modified_img.copy()
          modified_img = Image.fromarray(modified_img)
          modified_img.save(root+'/'+file[:-4]+'_modified.png')
          counter+=1
print('\rfinish!')

In [ ]:
%%bash

cd /content/dataset
find . -name '*labelIds.png' -type f -delete

# Reading the Data

In [ ]:
def read_files(img_dir, gt_dir, output_name):
    
    img_paths=[]
    gt_paths=[]

    for subdir in os.listdir(img_dir):
        
        imgs=[]
        gts=[]
        
        for file in os.listdir(img_dir+'/'+subdir):
            imgs.append(img_dir+'/'+subdir+'/'+file)
        
        for file in os.listdir(gt_dir+'/'+subdir):
                gts.append(gt_dir+'/'+subdir+'/'+file)

        assert len(imgs)==len(gts)

        imgs.sort()
        gts.sort()

        img_paths.extend(imgs)
        gt_paths.extend(gts)

    paths = (img_paths, gt_paths)

    return paths

In [ ]:
train_files = read_files(img_dir='/content/dataset/leftImg8bit/train', gt_dir='/content/dataset/gtFine/train', output_name='train')
test_files = read_files(img_dir='/content/dataset/leftImg8bit/test', gt_dir='/content/dataset/gtFine/test', output_name='test')
val_files = read_files(img_dir='/content/dataset/leftImg8bit/val', gt_dir='/content/dataset/gtFine/val', output_name='val')

# Data Augmentation

In [ ]:
def augment_data(aug, files, n_augmentation, augmented_img_path, augmented_gt_path):
    
    try:
        os.makedirs(augmented_img_path)
    except FileExistsError:
        pass

    try:
        os.makedirs(augmented_gt_path)
    except FileExistsError:
        pass

    
    if augmented_img_path[-1]!='/': augmented_img_path = augmented_img_path + '/'
    if augmented_gt_path[-1]!='/': augmented_gt_path = augmented_gt_path + '/'
    
    img_paths, gt_paths = files
    indices = np.random.randint(0,len(img_paths)-1, n_augmentation)


    for count, idx in enumerate(indices):

        img_path = img_paths[idx]
        gt_path = gt_paths[idx]

        img = np.array(Image.open(img_path))
        gt = np.expand_dims(np.array(Image.open(gt_path)), axis=0)

        imag_aug, gt_aug = aug(image=img, segmentation_maps=gt)

        imag_aug = Image.fromarray(imag_aug)
        gt_aug = Image.fromarray(gt_aug[0])

        imag_aug.save(augmented_img_path+img_path.split('/')[-1][:-4]+'_augmented{}.png'.format(count+1))
        gt_aug.save(augmented_gt_path+gt_path.split('/')[-1][:-4]+'_augmented{}.png'.format(count+1))

        print('\r', count+1, end='')

    print('\rfinish', end='')

In [ ]:
aug = iaa.Sequential([iaa.Crop(percent=(0, 0.4)),
                      iaa.LinearContrast((0.75, 1.25)),
                      iaa.Multiply((0.8, 1.2), per_channel=0.2),
                      iaa.AdditivePoissonNoise((10,20))
                      ], random_order=True)

files = train_files

n_augmentation = 1000

augmented_img_path = '/content/dataset/leftImg8bit/train/augmented'
augmented_gt_path =  '/content/dataset/gtFine/train/augmented'

augment_data(aug, files, n_augmentation, augmented_img_path, augmented_gt_path)

In [ ]:
train_files = read_files(img_dir='/content/dataset/leftImg8bit/train', gt_dir='/content/dataset/gtFine/train', output_name='train')
test_files = read_files(img_dir='/content/dataset/leftImg8bit/test', gt_dir='/content/dataset/gtFine/test', output_name='test')
val_files = read_files(img_dir='/content/dataset/leftImg8bit/val', gt_dir='/content/dataset/gtFine/val', output_name='val')

#Building the Datasets

In [ ]:
class Dataset:

    IMAGE_TRANSFORM = transforms.Compose([transforms.ToTensor()])
    LABEL_TRANSFORM = transforms.Compose([transforms.Lambda(lambda PIL_label: torch.IntTensor(np.array(PIL_label)))])

    def __init__(self, images, ground_truths):
        
        assert len(images)==len(ground_truths)

        self.images = images
        self.ground_truths = ground_truths
        self.samples = list(zip(self.images, self.ground_truths))


    def __len__(self):
      
        return len(self.samples)


    def __getitem__(self, index):

        if isinstance(index, int):
            image_path, gt_path = self.samples[index]
            image, gt = Image.open(image_path), Image.open(gt_path)
            return (self.IMAGE_TRANSFORM(image), self.LABEL_TRANSFORM(gt))

In [ ]:
train_set = Dataset(*train_files)
test_set = Dataset(*test_files)
val_set = Dataset(*val_files)

# Building the Model

## Segmentation Block

In [ ]:
class ASPPConv(nn.Sequential):
    
    def __init__(self, in_channels, out_channels, dilation):
        
        modules = [nn.Conv2d(in_channels, out_channels, 3, padding=dilation, dilation=dilation, bias=False),
                   nn.BatchNorm2d(out_channels),
                   nn.ReLU(inplace=True)]

        super(ASPPConv, self).__init__(*modules)




class ASPPPooling(nn.Sequential):

    def __init__(self, in_channels, out_channels):

        super(ASPPPooling, self).__init__(nn.AdaptiveAvgPool2d(1),
                                          nn.Conv2d(in_channels, out_channels, 1, bias=False),
                                          nn.BatchNorm2d(out_channels),
                                          nn.ReLU(inplace=True))


    def forward(self, x):

        size = x.shape[-2:]
        x = super(ASPPPooling, self).forward(x)
        return F.interpolate(x, size=size, mode='bilinear', align_corners=False)



class ASPP(nn.Module):

    def __init__(self, in_channels, atrous_rates):

        super(ASPP, self).__init__()

        out_channels = 256

        modules = []

        modules.append(nn.Sequential(nn.Conv2d(in_channels, out_channels, 1, bias=False),
                                     nn.BatchNorm2d(out_channels),
                                     nn.ReLU(inplace=True)))

        rate1, rate2, rate3 = tuple(atrous_rates)

        modules.append(ASPPConv(in_channels, out_channels, rate1))
        modules.append(ASPPConv(in_channels, out_channels, rate2))
        modules.append(ASPPConv(in_channels, out_channels, rate3))
        modules.append(ASPPPooling(in_channels, out_channels))

        self.convs = nn.ModuleList(modules)

        self.project = nn.Sequential(
            nn.Conv2d(5 * out_channels, out_channels, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Dropout(0.1),)


    def forward(self, x):
        res = []
        for conv in self.convs:
            res.append(conv(x))
        res = torch.cat(res, dim=1)
        return self.project(res)

In [ ]:
class DeepLabHeadV3Plus(nn.Module):


    def __init__(self, in_channels, low_level_channels, num_classes, aspp_dilate=[12, 24, 36]):

        super(DeepLabHeadV3Plus, self).__init__()

        self.project = nn.Sequential(nn.Conv2d(low_level_channels, 48, 1, bias=False),
                                     nn.BatchNorm2d(48),
                                     nn.ReLU(inplace=True))

        self.aspp = ASPP(in_channels, aspp_dilate)

        self.classifier = nn.Sequential(nn.Conv2d(304, 256, 3, padding=1, bias=False),
                                        nn.BatchNorm2d(256),
                                        nn.ReLU(inplace=True),
                                        nn.Conv2d(256, num_classes, 1))
        self._init_weight()


    def forward(self, feature):
        global aba
        aba = feature
        low_level_feature = self.project(feature['low_level'])
        output_feature = self.aspp(feature['out'])
        output_feature = F.interpolate(output_feature, size=low_level_feature.shape[2:], mode='bilinear', align_corners=False)
        return self.classifier(torch.cat([low_level_feature, output_feature], dim=1))
    

    def _init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)





class DeepLabHead(nn.Module):

    def __init__(self, in_channels, num_classes, aspp_dilate=[12, 24, 36]):

        super(DeepLabHead, self).__init__()

        self.classifier = nn.Sequential(ASPP(in_channels, aspp_dilate),
                                        nn.Conv2d(256, 256, 3, padding=1, bias=False),
                                        nn.BatchNorm2d(256),
                                        nn.ReLU(inplace=True),
                                        nn.Conv2d(256, num_classes, 1))
        
        self._init_weight()


    def forward(self, feature):
        return self.classifier( feature['out'] )


    def _init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

## trunk

## ResNet

In [ ]:
def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out




class Bottleneck(nn.Module):

    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
      
        super(Bottleneck, self).__init__()

        if norm_layer is None:
            norm_layer = nn.BatchNorm2d

        width = int(planes * (base_width / 64.)) * groups

        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride


    def forward(self, x):
        
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

In [ ]:
class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):

        super(ResNet, self).__init__()

        if norm_layer is None:
            norm_layer = nn.BatchNorm2d

        self._norm_layer = norm_layer

        self.inplanes = 64

        self.dilation = 1

        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]

        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups

        self.base_width = width_per_group

        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)

        self.bn1 = norm_layer(self.inplanes)

        self.relu = nn.ReLU(inplace=True)

        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 64, layers[0])

        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])

        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])

        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def forward(self, x):

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

## MobileNet

In [ ]:
def _make_divisible(v, divisor, min_value=None):

    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v


def fixed_padding(kernel_size, dilation):
  
    kernel_size_effective = kernel_size + (kernel_size - 1) * (dilation - 1)
    pad_total = kernel_size_effective - 1
    pad_beg = pad_total // 2
    pad_end = pad_total - pad_beg
    return (pad_beg, pad_end, pad_beg, pad_end) 



class ConvBNReLU(nn.Sequential):

    def __init__(self, in_planes, out_planes, kernel_size=3, stride=1, dilation=1, groups=1):
        
        super(ConvBNReLU, self).__init__(nn.Conv2d(in_planes, out_planes, kernel_size, stride, 0, dilation=dilation, groups=groups, bias=False),
                                         nn.BatchNorm2d(out_planes),
                                         nn.ReLU6(inplace=True))
        


class InvertedResidual(nn.Module):
  
    def __init__(self, inp, oup, stride, dilation, expand_ratio):
        
        super(InvertedResidual, self).__init__()
        
        self.stride = stride
        assert stride in [1, 2]

        hidden_dim = int(round(inp * expand_ratio))
        self.use_res_connect = self.stride == 1 and inp == oup

        layers = []
        if expand_ratio != 1:
            layers.append(ConvBNReLU(inp, hidden_dim, kernel_size=1))

        layers.extend([ConvBNReLU(hidden_dim, hidden_dim, stride=stride, dilation=dilation, groups=hidden_dim),
                       nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                       nn.BatchNorm2d(oup),])
        
        self.conv = nn.Sequential(*layers)

        self.input_padding = fixed_padding( 3, dilation )


    def forward(self, x):

        x_pad = F.pad(x, self.input_padding)
        if self.use_res_connect:
            return x + self.conv(x_pad)
        else:
            return self.conv(x_pad)



class MobileNetV2(nn.Module):

    def __init__(self, num_classes=1000, output_stride=8, width_mult=1.0, inverted_residual_setting=None, round_nearest=8):

        super(MobileNetV2, self).__init__()
        block = InvertedResidual
        input_channel = 32
        last_channel = 1280
        self.output_stride = output_stride
        current_stride = 1
        if inverted_residual_setting is None:
            inverted_residual_setting = [[1, 16, 1, 1],
                                         [6, 24, 2, 2],
                                         [6, 32, 3, 2],
                                         [6, 64, 4, 2],
                                         [6, 96, 3, 1],
                                         [6, 160, 3, 2],
                                         [6, 320, 1, 1],]

        if len(inverted_residual_setting) == 0 or len(inverted_residual_setting[0]) != 4:
            raise ValueError("inverted_residual_setting should be non-empty "
                             "or a 4-element list, got {}".format(inverted_residual_setting))

        input_channel = _make_divisible(input_channel * width_mult, round_nearest)
        self.last_channel = _make_divisible(last_channel * max(1.0, width_mult), round_nearest)
        features = [ConvBNReLU(3, input_channel, stride=2)]
        current_stride *= 2
        dilation=1
        previous_dilation = 1

        for t, c, n, s in inverted_residual_setting:
            output_channel = _make_divisible(c * width_mult, round_nearest)
            previous_dilation = dilation
            if current_stride == output_stride:
                stride = 1
                dilation *= s
            else:
                stride = s
                current_stride *= s
            output_channel = int(c * width_mult)

            for i in range(n):
                if i==0:
                    features.append(block(input_channel, output_channel, stride, previous_dilation, expand_ratio=t))
                else:
                    features.append(block(input_channel, output_channel, 1, dilation, expand_ratio=t))
                input_channel = output_channel

        features.append(ConvBNReLU(input_channel, self.last_channel, kernel_size=1))

        self.features = nn.Sequential(*features)

        self.classifier = nn.Sequential(nn.Dropout(0.2),
                                        nn.Linear(self.last_channel, num_classes),)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.zeros_(m.bias)

    def forward(self, x):
        x = self.features(x)
        x = x.mean([2, 3])
        x = self.classifier(x)
        return x


def mobilenet_v2(pretrained=False, progress=True, **kwargs):
  
    model = MobileNetV2(**kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls['mobilenet_v2'],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model

In [ ]:
class IntermediateLayerGetter(nn.ModuleDict):

    def __init__(self, model, return_layers):
        if not set(return_layers).issubset([name for name, _ in model.named_children()]):
            raise ValueError("return_layers are not present in model")

        orig_return_layers = return_layers
        return_layers = {k: v for k, v in return_layers.items()}
        layers = OrderedDict()
        for name, module in model.named_children():
            layers[name] = module
            if name in return_layers:
                del return_layers[name]
            if not return_layers:
                break

        super(IntermediateLayerGetter, self).__init__(layers)
        self.return_layers = orig_return_layers

    def forward(self, x):
        out = OrderedDict()
        for name, module in self.named_children():
            x = module(x)
            if name in self.return_layers:
                out_name = self.return_layers[name]
                out[out_name] = x
        return out

In [ ]:
def _resnet(arch, block, layers, pretrained, progress, **kwargs):

    model = ResNet(block, layers, **kwargs)

    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
        
    return model

In [ ]:
def resnet50(pretrained=False, progress=True, **kwargs):
    
    return _resnet('resnet50', Bottleneck, [3, 4, 6, 3], pretrained, progress, **kwargs)


def resnet101(pretrained=False, progress=True, **kwargs):

    return _resnet('resnet101', Bottleneck, [3, 4, 23, 3], pretrained, progress, **kwargs)

In [ ]:
resnet_dict = {
               'resnet101':resnet101,
               'resnet50':resnet50,}

model_urls = {'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
              'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth'}


## Attention

In [ ]:
class Attention(nn.Module):

    def __init__(self, in_channels):

        super(Attention, self).__init__()

        self.attention_mask = nn.Sequential(nn.ConvTranspose2d(in_channels=in_channels, out_channels=512, kernel_size=(6,6), stride=2, padding=2),
                                            nn.BatchNorm2d(num_features=512),
                                            nn.LeakyReLU(negative_slope=0.01),
                                            nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=(6,6), stride=2, padding=2),
                                            nn.BatchNorm2d(num_features=256),
                                            nn.LeakyReLU(negative_slope=0.01),
                                            nn.ConvTranspose2d(in_channels=256, out_channels=64, kernel_size=(6,6), stride=2, padding=2),
                                            nn.BatchNorm2d(num_features=64),
                                            nn.LeakyReLU(negative_slope=0.01),
                                            nn.ConvTranspose2d(in_channels=64, out_channels=1, kernel_size=(5,5), stride=1, padding=2),
                                            nn.Sigmoid()
                                            )


    def forward(self, x):
      
      return self.attention_mask(x) 

## Network Architecture

In [ ]:
def _segm_resnet(segmentation_name, backbone_name, num_classes, output_stride, pretrained_backbone):

    if output_stride==8:
        replace_stride_with_dilation=[False, True, True]
        aspp_dilate = [12, 24, 36]
    else:
        replace_stride_with_dilation=[False, False, True]
        aspp_dilate = [6, 12, 18]

    backbone = resnet_dict[backbone_name](
        pretrained=pretrained_backbone,
        replace_stride_with_dilation=replace_stride_with_dilation)
    
    inplanes = 2048
    low_level_planes = 256

    return_layers = None
    classifier = None

    if segmentation_name=='deeplabv3+':
        return_layers = {'layer4': 'out', 'layer1': 'low_level'}
        classifier = DeepLabHeadV3Plus(inplanes, low_level_planes, num_classes, aspp_dilate)

    elif segmentation_name=='deeplabv3':
        return_layers = {'layer4': 'out'}
        classifier = DeepLabHead(inplanes , num_classes, aspp_dilate)

    else:
        raise NotImplementedError

    backbone = IntermediateLayerGetter(backbone, return_layers=return_layers)

    return backbone, classifier



def _segm_mobilenet(name, backbone_name, num_classes, output_stride, pretrained_backbone):
    
    if output_stride==8:
        aspp_dilate = [12, 24, 36]
    else:
        aspp_dilate = [6, 12, 18]

    backbone = mobilenet_v2(pretrained=pretrained_backbone, output_stride=output_stride)
    
    backbone.low_level_features = backbone.features[0:4]
    backbone.high_level_features = backbone.features[4:-1]
    backbone.features = None
    backbone.classifier = None

    inplanes = 320
    low_level_planes = 24
    
    return_layers = None
    classifier = None
    if name=='deeplabv3+':
        return_layers = {'high_level_features': 'out', 'low_level_features': 'low_level'}
        classifier = DeepLabHeadV3Plus(inplanes, low_level_planes, num_classes, aspp_dilate)
    elif name=='deeplabv3':
        return_layers = {'high_level_features': 'out'}
        classifier = DeepLabHead(inplanes , num_classes, aspp_dilate)
    backbone = IntermediateLayerGetter(backbone, return_layers=return_layers)

    return (backbone, classifier)

In [ ]:
class InitStage(nn.Module):

    def __init__(self, scale, segmentation_name, backbone_name, num_classes, pretrained_backbone=True, freeze_trunk=False, freeze_seg=False):

        assert scale>0

        super(InitStage, self).__init__()

        trunk_loader = None
        if backbone_name=='mobilenetv2':
            trunk_loader = _segm_mobilenet
        elif backbone_name.startswith('resnet'):
            trunk_loader = _segm_resnet
        else:
            raise NotImplementedError
        
        self.trunk, self.segmentation = trunk_loader(segmentation_name, backbone_name, num_classes, output_stride=8, pretrained_backbone=True)
        
        if freeze_trunk:
            self.freeze_trunk()

        if freeze_seg:
            self.freeze_seg()
        
        self.scale = scale


    
    def forward(self, x):

        input_shape = x.shape[-2:]
        x = F.interpolate(input=x, scale_factor=self.scale, mode='bilinear')
        x = self.trunk(x)
        x = self.segmentation(x)
        x = F.interpolate(x, size=input_shape, mode='bilinear', align_corners=False)
        x = F.sigmoid(x)

        return x


    def freeze_trunk(self):
        for p in self.trunk.parameters():
            p.requires_grad = False


    def freeze_seg(self):
        for p in self.segmentation.parameters():
            p.requires_grad = False


    def unfreeze_trunk(self):
        for p in self.trunk.parameters():
            p.requires_grad = True


    def unfreeze_seg(self):
        for p in self.segmentation.parameters():
            p.requires_grad = True
          

    def get_trainable_param(self): 
        return filter(lambda p: p.requires_grad, self.parameters())

In [ ]:
class Stage(nn.Module):

    def __init__(self, scale, segmentation_name, backbone_name, num_classes, freeze_trunk=False, freeze_seg=False):

        super(Stage, self).__init__()

        trunk_loader = None
        trunk_out_channels = None
        if backbone_name=='mobilenetv2':
            trunk_loader = _segm_mobilenet
            trunk_out_channels = 320
        elif backbone_name.startswith('resnet'):
            trunk_loader = _segm_resnet
            trunk_out_channels = 2048
        else:
            raise NotImplementedError
        
        self.trunk, self.segmentation = trunk_loader(segmentation_name, backbone_name, num_classes, output_stride=8, pretrained_backbone=True)
        
        if freeze_trunk:
            self.freeze_trunk()

        if freeze_seg:
            self.freeze_seg()

        self.attention = Attention(in_channels=trunk_out_channels)

        self.scale = scale


    def forward(self, x, prev_stage_output):

        x = F.interpolate(input=x, scale_factor=self.scale, mode='bilinear')
        input_shape = x.shape[2:]

        trunk_output = self.trunk(x)

        segmentation_output = self.segmentation(trunk_output)
        segmentation_output = F.interpolate(segmentation_output, size=input_shape, mode='bilinear', align_corners=False)

        attention_output = self.attention(trunk_output['out'])

        attention_output = F.interpolate(attention_output, size=input_shape, mode='bilinear', align_corners=False)

        new = attention_output * segmentation_output

        new = F.interpolate(input=new, scale_factor=1/self.scale, mode='bilinear')

        scaled_attention = F.interpolate(input=attention_output, size=prev_stage_output.shape[2:], mode='bilinear')
        prev = (1-scaled_attention) * prev_stage_output

        output = F.sigmoid(new + prev)

        return output

    
    def freeze_trunk(self):
        for p in self.trunk.parameters():
            p.requires_grad = False


    def freeze_seg(self):
        for p in self.segmentation.parameters():
            p.requires_grad = False

  
    def freeze_attention(self):
        for p in self.attention.parameters():
            p.requires_grad = False
      
    
    def unfreeze_trunk(self):
        for p in self.trunk.parameters():
            p.requires_grad = True


    def unfreeze_seg(self):
        for p in self.segmentation.parameters():
            p.requires_grad = True


    def unfreeze_attention(self):
        for p in self.attention.parameters():
            p.requires_grad = True


    def get_trainable_param(self): 

        return filter(lambda p: p.requires_grad, self.parameters())

In [ ]:
class HierachicalModel(nn.Module):

    def __init__(self, n_stages, scales, segmentation, backbone, n_classes=20):

        super(HierachicalModel, self).__init__()

        assert type(n_stages)==int and n_stages>0
        assert (type(scales)==list or type(scales)==tuple) and len(scales)==n_stages

        self.n_classes = n_classes
        self.init_stage = InitStage(scale=scales[0], segmentation_name=segmentation, backbone_name=backbone, num_classes=self.n_classes)
        self.next_stages = nn.ModuleList([Stage(scale=scale, segmentation_name=segmentation, backbone_name=backbone, num_classes=self.n_classes) for scale in scales[1:]])

        # self.sementic_head = nn.Sequential(nn.Conv2d(in_channels=1, out_channels=96, kernel_size=(3,3), stride=4, padding=4),
        #                                    nn.BatchNorm2d(100),
        #                                    nn.ReLU(),
        #                                    nn.Conv2d(in_channels=3, out_channels=96, kernel_size=(3,3), stride=4, padding=4),
        #                                    nn.BatchNorm2d(100),
        #                                    nn.ReLU(),
        #                                    nn.Conv2d(in_channels=3, out_channels=self.N_CLASSES, kernel_size=(1,1), stride=4, padding=4))
        


    def forward(self, x):

        output = self.init_stage(x)

        for stage in self.next_stages:
            output = stage(x, output)

        # output = sementic_head(output)

        return output

        
    def get_trainable_param(self): 

        return filter(lambda p: p.requires_grad, self.parameters())

# Metrics

## Implementing the Loss Function

In [ ]:
class RMILoss(nn.Module):

    _CLIP_MIN = 1e-6
    _CLIP_MAX = 1.0
    _POS_ALPHA = 5e-4
    _IS_SUM = 1


    def __init__(self,
                 num_classes=21,
                 rmi_radius=3,
                 rmi_pool_way=0,
                 rmi_pool_size=3,
                 rmi_pool_stride=3,
                 loss_weight_lambda=0.5,
                 lambda_way=1,):
   
        super(RMILoss, self).__init__()
  
        assert rmi_radius<=10 and type(rmi_radius)==int
        assert rmi_pool_way<=3 and type(rmi_pool_way)==int
        assert rmi_pool_size == rmi_pool_stride

        self.num_classes = num_classes
        self.rmi_radius = rmi_radius
        self.rmi_pool_way = rmi_pool_way
        self.rmi_pool_size = rmi_pool_size
        self.rmi_pool_stride = rmi_pool_stride
        self.weight_lambda = loss_weight_lambda
        self.lambda_way = lambda_way

        self.half_d = self.rmi_radius * self.rmi_radius
        self.d = 2 * self.half_d
        self.kernel_padding = self.rmi_pool_size//2
        self.ignore_index = 255




    def forward(self, logits_4D, labels_4D):
        loss = self.forward_sigmoid(logits_4D, labels_4D)
        #loss = self.forward_softmax_sigmoid(logits_4D, labels_4D)
        return loss



    def forward_softmax_sigmoid(self, logits_4D, labels_4D):

        normal_loss = F.cross_entropy(input=logits_4D,
                                        target=labels_4D.long(),
                                        ignore_index=self.ignore_index,
                                        reduction='mean')

        label_mask_3D = labels_4D < self.num_classes
        valid_onehot_labels_4D = F.one_hot(labels_4D.long() * label_mask_3D.long(), num_classes=self.num_classes).float()
        label_mask_3D = label_mask_3D.float()
        valid_onehot_labels_4D = valid_onehot_labels_4D * label_mask_3D.unsqueeze(dim=3)
        valid_onehot_labels_4D = valid_onehot_labels_4D.permute(0, 3, 1, 2).requires_grad_(False)
        probs_4D = F.sigmoid(logits_4D) * label_mask_3D.unsqueeze(dim=1)
        probs_4D = probs_4D.clamp(min=self._CLIP_MIN, max=self._CLIP_MAX)
        rmi_loss = self.rmi_lower_bound(valid_onehot_labels_4D, probs_4D)

        final_loss = (self.weight_lambda * normal_loss + rmi_loss * (1 - self.weight_lambda) if self.lambda_way
                        else normal_loss + rmi_loss * self.weight_lambda)

        return final_loss


    def forward_sigmoid(self, logits_4D, labels_4D):

        label_mask_3D = labels_4D < self.num_classes

        valid_onehot_labels_4D = F.one_hot(labels_4D.long() * label_mask_3D.long(), num_classes=self.num_classes).float()
        label_mask_3D = label_mask_3D.float()
        label_mask_flat = label_mask_3D.view([-1, ])
        valid_onehot_labels_4D = valid_onehot_labels_4D * label_mask_3D.unsqueeze(dim=3)
        valid_onehot_labels_4D.requires_grad_(False)

        valid_onehot_label_flat = valid_onehot_labels_4D.view([-1, self.num_classes]).requires_grad_(False)
        logits_flat = logits_4D.permute(0, 2, 3, 1).contiguous().view([-1, self.num_classes])

        valid_pixels = torch.sum(label_mask_flat)
        binary_loss = F.binary_cross_entropy_with_logits(logits_flat,
                                                            target=valid_onehot_label_flat,
                                                            weight=label_mask_flat.unsqueeze(dim=1),
                                                            reduction='sum')
        bce_loss = torch.div(binary_loss, valid_pixels + 1.0)

        probs_4D = logits_4D.sigmoid() * label_mask_3D.unsqueeze(dim=1) + self._CLIP_MIN
        valid_onehot_labels_4D = valid_onehot_labels_4D.permute(0, 3, 1, 2).requires_grad_(False)

        rmi_loss = self.rmi_lower_bound(valid_onehot_labels_4D, probs_4D)

        final_loss = (self.weight_lambda * bce_loss + rmi_loss * (1 - self.weight_lambda) if self.lambda_way
                else bce_loss + rmi_loss * self.weight_lambda)

        return final_loss


    def rmi_lower_bound(self, labels_4D, probs_4D):

        assert labels_4D.size() == probs_4D.size()

        p, s = self.rmi_pool_size, self.rmi_pool_stride
        if self.rmi_pool_stride > 1:
            if self.rmi_pool_way == 0:
                labels_4D = F.max_pool2d(labels_4D, kernel_size=p, stride=s, padding=self.kernel_padding)
                probs_4D = F.max_pool2d(probs_4D, kernel_size=p, stride=s, padding=self.kernel_padding)
            elif self.rmi_pool_way == 1:
                labels_4D = F.avg_pool2d(labels_4D, kernel_size=p, stride=s, padding=self.kernel_padding)
                probs_4D = F.avg_pool2d(probs_4D, kernel_size=p, stride=s, padding=self.kernel_padding)
            elif self.rmi_pool_way == 2:
                shape = labels_4D.size()
                new_h, new_w = shape[2] // s, shape[3] // s
                labels_4D = F.interpolate(labels_4D, size=(new_h, new_w), mode='nearest')
                probs_4D = F.interpolate(probs_4D, size=(new_h, new_w), mode='bilinear', align_corners=True)
            else:
                raise NotImplementedError("Pool way of RMI is not defined!")
        label_shape = labels_4D.size()
        n, c = label_shape[0], label_shape[1]
        la_vectors, pr_vectors = self.map_get_pairs(labels_4D, probs_4D, radius=self.rmi_radius, is_combine=0)
        la_vectors = la_vectors.view([n, c, self.half_d, -1]).type(torch.cuda.DoubleTensor).requires_grad_(False)
        pr_vectors = pr_vectors.view([n, c, self.half_d, -1]).type(torch.cuda.DoubleTensor)
        diag_matrix = torch.eye(self.half_d).unsqueeze(dim=0).unsqueeze(dim=0)
        la_vectors = la_vectors - la_vectors.mean(dim=3, keepdim=True)
        la_cov = torch.matmul(la_vectors, la_vectors.transpose(2, 3))
        pr_vectors = pr_vectors - pr_vectors.mean(dim=3, keepdim=True)
        pr_cov = torch.matmul(pr_vectors, pr_vectors.transpose(2, 3))
        pr_cov_inv = torch.inverse(pr_cov + diag_matrix.type_as(pr_cov) * self._POS_ALPHA)
        la_pr_cov = torch.matmul(la_vectors, pr_vectors.transpose(2, 3))
        appro_var = la_cov - torch.matmul(la_pr_cov.matmul(pr_cov_inv), la_pr_cov.transpose(-2, -1))
        rmi_now = 0.5 * self.log_det_by_cholesky(appro_var + diag_matrix.type_as(appro_var) * self._POS_ALPHA)
        rmi_per_class = rmi_now.view([-1, self.num_classes]).mean(dim=0).float()

        rmi_per_class = torch.div(rmi_per_class, float(self.half_d))
        rmi_loss = torch.sum(rmi_per_class) if self._IS_SUM else torch.mean(rmi_per_class)

        return rmi_loss



    def map_get_pairs(aelf, labels_4D, probs_4D, radius=3, is_combine=True):

        h, w = labels_4D.size()[2:4]

        new_h, new_w = h-(radius-1), w-(radius-1)
    
        la_ns=[]
        pr_ns=[]

        for y in range(0, radius):
            for x in range(0, radius):
                la_now = labels_4D[:, :, y:y+new_h, x:x+new_w]
                pr_now = probs_4D[:, :, y:y+new_h, x:x+new_w]
                la_ns.append(la_now)
                pr_ns.append(pr_now)

        if is_combine:
            pair_ns = la_ns + pr_ns
            p_vectors = torch.stack(pair_ns, dim=2)
            return p_vectors
        
        else:
            la_vectors = torch.stack(la_ns, dim=2)
            pr_vectors = torch.stack(pr_ns, dim=2)
            return la_vectors, pr_vectors


    def log_det_by_cholesky(self, matrix):

      chol = torch.cholesky(matrix)
      return 2.0 * torch.sum(torch.log(torch.diagonal(chol, dim1=-2, dim2=-1) + 1e-8), dim=-1)



    def batch_cholesky_inverse(self, matrix):

      chol_low = torch.cholesky(matrix, upper=False)
      chol_low_inv = self.batch_low_tri_inv(chol_low)
      return torch.matmul(chol_low_inv.transpose(-2, -1), chol_low_inv)



    def batch_low_tri_inv(self, L):

      n = L.shape[-1]
      invL = torch.zeros_like(L)
      for j in range(0, n):
        invL[..., j, j] = 1.0 / L[..., j, j]
        for i in range(j + 1, n):
          S = 0.0
          for k in range(0, i + 1):
            S = S - L[..., i, k] * invL[..., k, j].clone()
          invL[..., i, j] = S / L[..., i, i]
      return invL

## Other Metrics

In [ ]:
class Metrics:

    ID2LABEL_NAME = {}
    for label, info in label2info.items():
        ID2LABEL_NAME[info.id] = label

    def mean_pixel_accuracy(pred, gt):

        gt = np.array(gt.to('cpu'))
        pred = np.array(pred.to('cpu'))

        unique, counts = np.unique(gt, return_counts=True)
        class_count = dict(zip(unique, counts))

        mean_pixel_accuracy = {}
        equality = (pred==gt)
        values = np.array([pred==id for id in class_count.keys()])
        correct_detection = values * equality
        for idx, (id, count) in enumerate(class_count.items()):
            mean_pixel_accuracy[Metrics.ID2LABEL_NAME[id]] = np.sum(correct_detection[idx])/count
        return mean_pixel_accuracy


    def IOU_for_classes(pred, gt):

        gt = np.array(gt.to('cpu'))
        pred = np.array(pred.to('cpu'))

        unique, counts = np.unique(gt, return_counts=True)
        class_count = dict(zip(unique, counts))

        IOU_values = {}

        pred_equal_to_id = np.array([pred==id for id in class_count.keys()])
        gt_equal_to_id = np.array([gt==id for id in class_count.keys()])

        intersection = gt_equal_to_id * pred_equal_to_id
        union = gt_equal_to_id + pred_equal_to_id

        for idx, (id, count) in enumerate(class_count.items()):
            IOU_values[Metrics.ID2LABEL_NAME[id]] = np.sum(intersection[idx])/np.sum(union[idx])
        
        return IOU_values

        
    def mean_IOU(pred, gt):

        return np.mean(list(Metrics.IOU_for_classes(pred, gt).values()))
    

    def pixel_accuracy(pred, gt):

        assert pred.shape==gt.shape

        total_pixels = torch.numel(pred)
        correctly_classified_pixels = torch.sum(pred==gt).item()

        return correctly_classified_pixels / total_pixels


# Trainer

In [ ]:
class Trainer():

    N_CLASSES = 20
    
    def __init__(self,
                 model,
                 optimizer,
                 device,
                 log_dir=time.strftime("/content/runs/RUN-%Y-%m-%d %H:%M:%S", time.gmtime())):

        self.device = device
        self.model = model.to(self.device)
        self.optimizer = optimizer
        self.loss = RMILoss(num_classes=self.N_CLASSES)
        self.writer = SummaryWriter(log_dir=log_dir)
        



    def train_model(self, train_loader, epoch_number, print_results=True):
        
        assert epoch_number > 0 and type(epoch_number)==int

        self.model.train()

        loss_sum = 0
        accuracy_sum = 0
        mean_IOU_sum = 0

        n_batches = int(len(train_loader.dataset) / train_loader.batch_size)
        
        for n_batch, (data, target) in enumerate(train_loader):
            
            if len(data)==1: continue
            
            if self.device.type=='cuda':
                torch.cuda.empty_cache()
            
            data = data.to(self.device)
            target = target.to(self.device).long()

            self.optimizer.zero_grad()

            output = self.model(data)

            loss = self.loss(output, target)
            average_batch_loss = loss.item()
            loss_sum += average_batch_loss
            loss.backward()
            self.optimizer.step()

            batch_pixel_accuracy = Metrics.pixel_accuracy(self.__get_predicted_labels_from_output(output), target)
            accuracy_sum += batch_pixel_accuracy
            
            batch_mean_IOU = Metrics.mean_IOU(self.__get_predicted_labels_from_output(output), target)
            mean_IOU_sum += batch_mean_IOU

            if print_results:
                print('\rTraining Batch {}/{}:\tAverage Loss: {:.3f}\tPixel Accuracy: {:.3f}\tMean IOU: {:.3f}\t\tAverage IOU: {:.3f}'.format(n_batch+1, 
                                                                                                                                              n_batches, 
                                                                                                                                              average_batch_loss, 
                                                                                                                                              batch_pixel_accuracy, 
                                                                                                                                              batch_mean_IOU, 
                                                                                                                                              mean_IOU_sum/(n_batch+1)), end='')
                if self.device.type=='cuda':
                    print('\tGPU Usage: {:.1f}%'.format(GPU.getGPUs()[0].memoryUtil*100), end='')
                    
        
        average_loss = loss_sum / n_batches
        average_accuracy = accuracy_sum / n_batches
        average_mean_IOU = mean_IOU_sum / n_batches

        self.writer.add_scalar('Loss/train', average_loss, epoch_number)
        self.writer.add_scalar('pixel_accuracy/train', average_accuracy, epoch_number)
        self.writer.add_scalar('mean_IOU/train', average_mean_IOU, epoch_number)

        if print_results:
            print('\r',end='')

        return (average_loss, average_accuracy, average_mean_IOU)


         
    def test_model(self, test_loader, epoch_number, print_results=True):

        assert epoch_number > 0 and type(epoch_number)==int
        
        self.model.eval()

        loss_sum = 0
        accuracy_sum = 0
        mean_IOU_sum = 0

        n_batches = int(len(test_loader.dataset) / test_loader.batch_size)

        with torch.no_grad():
          for n_batch, (data, target) in enumerate(test_loader):
              
              if len(data)==1: continue

              if self.device.type=='cuda':
                  torch.cuda.empty_cache()
              
              data = data.to(self.device)
              target = target.to(self.device).long()
              output = self.model(data)
              loss = self.loss(output, target)

              average_batch_loss = loss.item()
              loss_sum += average_batch_loss

              batch_pixel_accuracy = Metrics.pixel_accuracy(self.__get_predicted_labels_from_output(output), target)
              accuracy_sum += batch_pixel_accuracy
              
              batch_mean_IOU = Metrics.mean_IOU(self.__get_predicted_labels_from_output(output), target)
              mean_IOU_sum += batch_mean_IOU

              if print_results:
                  print('\rTesting Batch {}/{}:\tTotal Batch Loss Loss: {:.3f}\tPixel Accuracy: {:.3f}\tMean IOU: {:.3f}\t\tAverage IOU: {:.3f}'.format(n_batch+1, 
                                                                                                                                                        n_batches, 
                                                                                                                                                        average_batch_loss, 
                                                                                                                                                        batch_pixel_accuracy, 
                                                                                                                                                        batch_mean_IOU, 
                                                                                                                                                        mean_IOU_sum/(n_batch+1)), end='')
                  if self.device.type=='cuda':
                      print('\tGPU Usage: {:.1f}%'.format(GPU.getGPUs()[0].memoryUtil*100), end='')
                 
        average_loss = loss_sum / n_batches 
        average_accuracy = accuracy_sum / n_batches
        average_mean_IOU = mean_IOU_sum / n_batches

        self.writer.add_scalar('Loss/test', average_loss, epoch_number)
        self.writer.add_scalar('pixel_accuracy/test', average_accuracy, epoch_number)
        self.writer.add_scalar('mean_IOU/test', average_mean_IOU, epoch_number)

        if print_results:
            print('\r',end='')

        return (average_loss, average_accuracy, average_mean_IOU)


    def get_model_output(self, input):

        return self.model(input.to(self.device))

    
    def __get_predicted_labels_from_output(self, output):

        return torch.argmax(output, -3)

# Visualization

In [ ]:
class Visualization:

    ID2COLOR = {label.id : np.array(label.color) for label in label2info.values()}

    def get_color_array(network_output):

        assert len(network_output.size())==3
        ids = torch.argmax(network_output.to('cpu'),0)

        channel1 = np.expand_dims(np.vectorize(lambda key:Visualization.ID2COLOR[key][0])(ids), -1)
        channel2 = np.expand_dims(np.vectorize(lambda key:Visualization.ID2COLOR[key][1])(ids), -1)
        channel3 = np.expand_dims(np.vectorize(lambda key:Visualization.ID2COLOR[key][2])(ids), -1)

        return np.concatenate((channel1, channel2, channel3), -1).astype(np.uint8)


    def visualize_label(label):

        assert len(label.size())==2

        channel1 = np.expand_dims(np.vectorize(lambda key:Visualization.ID2COLOR[key][0])(label), -1)
        channel2 = np.expand_dims(np.vectorize(lambda key:Visualization.ID2COLOR[key][1])(label), -1)
        channel3 = np.expand_dims(np.vectorize(lambda key:Visualization.ID2COLOR[key][2])(label), -1)

        return Image.fromarray(np.concatenate((channel1, channel2, channel3), -1).astype(np.uint8))



    def get_color_label(network_output, step=None):

        return Image.fromarray(Visualization.get_color_array(network_output))


    def visualize_in_tb(image_arr, log_dir, name=time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())):

        with SummaryWriter() as writer:
            writer.add_image(name, image_arr, step)
        return 1

# Training

## Data Loaders

In [ ]:
BATCH_SIZE = 5

train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True)

## Loading Pre-Trained Parts

In [ ]:
def load_model(directory):
        
    saved_file = open(directory,"rb")
    return pickle.load(saved_file)


PATH = '/content/drive/My Drive/DL_Project/initial_2.pickle'

model = load_model(scale_2_path) 

## Training the Initial Stages

In [ ]:
model=InitStage(scale=0.5, segmentation_name='deeplabv3+', backbone_name='mobilenetv2', num_classes=20, freeze_trunk=False, freeze_seg=False)

In [ ]:
model.unfreeze_trunk()
model.unfreeze_seg()

In [ ]:
LEARNING_RATE = 1e-4
USE_GPU = True

device = torch.device("cuda:0" if torch.cuda.is_available() and USE_GPU else "cpu")
optimizer = optim.Adam(model.get_trainable_param(), lr=LEARNING_RATE)

trainer = Trainer(model, optimizer, device)

In [ ]:
EPOCHS = 20
LOG_EVERY = 1

train_results = None

for epoch in range(1, EPOCHS+1):

    print_results=False
    if not epoch%LOG_EVERY:
        print_results=True
        print("\nEpoch {}:".format(epoch))  

    train_results = trainer.train_model(train_loader=train_loader, epoch_number=epoch)
    test_results = trainer.test_model(test_loader=val_loader, epoch_number=epoch)

    if USE_GPU:
        torch.cuda.empty_cache()
    
    if print_results:
        print('\tAverage Loss: {:.3f}\tPixel Accuracy: {:.3f}\t Mean IOU: {:.3f}'.format(*train_results))
        print('\tAverage Loss: {:.3f}\tPixel Accuracy: {:.3f}\t Mean IOU: {:.3f}'.format(*test_results))  

# Training the Whole Model

In [ ]:
BATCH_SIZE = 6

train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
def load_model(directory):
        
    saved_file = open(directory,"rb")
    return pickle.load(saved_file)


PATH_SCALE_05 = '/content/drive/My Drive/DL_Project/best_model/model_scale_05.pickle'
PATH_SCALE_1 = '/content/drive/My Drive/DL_Project/best_model/model_scale_1.pickle'
PATH_SCALE_2 = '/content/drive/My Drive/DL_Project/best_model/model_scale_2.pickle'

initial_stage_scale_05 = load_model(PATH_SCALE_05) 
initial_stage_scale_1 = load_model(PATH_SCALE_1) 
initial_stage_scale_2 = load_model(PATH_SCALE_2) 

In [ ]:
model = HierachicalModel(n_stages=2, scales=(1, 0.5), segmentation='deeplabv3+', backbone='mobilenetv2')

model.init_stage.trunk = initial_stage_scale_1.trunk
model.init_stage.segmentation = initial_stage_scale_1.segmentation

model.next_stages[0].trunk = initial_stage_scale_05.trunk
model.next_stages[0].segmentation = initial_stage_scale_05.segmentation

In [ ]:
model.init_stage.freeze_trunk()
model.init_stage.freeze_seg()

model.next_stages[0].freeze_trunk()
model.next_stages[0].freeze_seg()

In [ ]:
LEARNING_RATE = 1e-4
USE_GPU = True

device = torch.device("cuda:0" if torch.cuda.is_available() and USE_GPU else "cpu")
optimizer = optim.Adam(model.get_trainable_param(), lr=LEARNING_RATE)

trainer = Trainer(model, optimizer, device)

In [ ]:
trainer.loss = nn.CrossEntropyLoss()

In [ ]:
EPOCHS = 20
LOG_EVERY = 1

train_results = None

for epoch in range(1, EPOCHS+1):

    print_results=False
    if not epoch%LOG_EVERY:
        print_results=True
        print("\nEpoch {}:".format(epoch))

    if epoch==6:
        model.init_stage.freeze_attention()
        model.init_stage.unfreeze_trunk()
        model.init_stage.unfreeze_seg()

        model.next_stages[0].freeze_attention()
        model.next_stages[0].freeze_trunk()
        model.next_stages[0].freeze_seg()

    
    if epoch==8:
        model.init_stage.freeze_attention()
        model.init_stage.unfreeze_trunk()
        model.init_stage.unfreeze_seg()

        model.next_stages[0].freeze_attention()
        model.next_stages[0].freeze_trunk()
        model.next_stages[0].freeze_seg()

    if epoch==10:
        model.init_stage.unfreeze_attention()
        model.init_stage.freeze_trunk()
        model.init_stage.freeze_seg()

        model.next_stages[0].unfreeze_attention()
        model.next_stages[0].freeze_trunk()
        model.next_stages[0].freeze_seg()


    train_results = trainer.train_model(train_loader=train_loader, epoch_number=epoch)
    test_results = trainer.test_model(test_loader=val_loader, epoch_number=epoch)

    if USE_GPU:
        torch.cuda.empty_cache()
    
    if print_results:
        print('\tAverage Loss: {:.3f}\tPixel Accuracy: {:.3f}\t Mean IOU: {:.3f}'.format(*train_results))
        print('\tAverage Loss: {:.3f}\tPixel Accuracy: {:.3f}\t Mean IOU: {:.3f}'.format(*test_results))  

# Visualizing the Results

In [ ]:
%tensorboard --logdir {logs_base_dir}